In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("/content/canonical_checkpoint_features_with_soh.parquet")

df = df.sort_values(["asset_id", "cycle_id"]).reset_index(drop=True)

df.head()


,asset_id,cycle_id,V_mean,V_std,V_min,V_max,V_range,dV_dt_mean,dV_dt_max,T_mean,...,dV_dt_base,T_delta_base,duration_base,use_dV_dt,V_range_norm,T_delta_norm,duration_norm,dV_dt_norm,Degradation_Index,SOH_proxy
0,0,0,3.735076,0.245422,2.699819,4.191235,1.491416,-35.948592,38.111247,40.419340,...,34.655228,1.335614,0.038896,True,0.692878,0.669490,0.729033,0.741807,0.708302,0.492480
1,0,1,3.735681,0.244972,2.699859,4.192679,1.492819,-36.342517,34.652440,40.327115,...,34.655228,1.335614,0.038896,True,0.693348,0.688454,0.722873,0.693107,0.699446,0.496861
2,0,10,3.739525,0.240002,2.699924,4.192582,1.492658,-38.022390,41.584546,40.363359,...,34.655228,1.335614,0.038896,True,0.693295,0.693133,0.699661,0.788435,0.718631,0.487419
3,0,11,3.739852,0.239651,2.699803,4.192502,1.492699,-38.328526,34.646588,40.433203,...,34.655228,1.335614,0.038896,True,0.693308,0.711655,0.697407,0.693023,0.698848,0.497158
4,0,12,3.740231,0.238893,2.699924,4.192462,1.492538,-38.309224,34.655228,40.393250,...,34.655228,1.335614,0.038896,True,0.693254,0.702437,0.695238,0.693147,0.696019,0.498566


In [ ]:
def split_by_time(group, train_frac=0.7, val_frac=0.15):
    n = len(group)
    train_end = int(n * train_frac)
    val_end = int(n * (train_frac + val_frac))

    split = np.empty(n, dtype=object)
    split[:train_end] = "train"
    split[train_end:val_end] = "val"
    split[val_end:] = "test"

    group = group.copy()
    group["split"] = split
    return group


In [ ]:
df = (
    df.groupby("asset_id", group_keys=False)
      .apply(split_by_time)
      .reset_index(drop=True)
)

df["split"].value_counts(normalize=True)


/tmp/ipython-input-1574791291.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(split_by_time)


,proportion
split,
train,0.699958
test,0.150038
val,0.150004


In [ ]:
df.groupby(["asset_id", "split"]).size().unstack(fill_value=0)


split,test,train,val
asset_id,,,
0,12,54,12
1,11,51,11
1.RFUD_RW9_10_11_12,16698,77923,16698
2,12,53,11
3,8,32,7
4,7,32,7
5,7,32,7
6,12,53,12
7,12,53,11


In [ ]:
df.groupby(["asset_id", "split"])["cycle_id"].agg(["min", "max"])


min         max
asset_id            split                        
0                   test           69           9
                    train           0          57
                    val            58          68
1                   test           65           9
                    train           0          54
                    val            55          64
1.RFUD_RW9_10_11_12 test    RW9_39952   RW9_99997
                    train      RW10_1  RW12_82558
                    val    RW12_82562    RW9_3995
2                   test           67           9
                    train           0          56
                    val            57          66
3                   test           44           9
                    train           0          37
                    val            38          43
4                   test           44           9
                    train           0          37
                    val            38          43
5                   test           44           9
                    train           0          37
                    val            38          43
6                   test           68           9
                    train           0          56
                    val            57          67
7                   test           67           9
                    train           0          56
                    val            57          66
RFUD_RW1_2_7_8      test          812         999
                    train           0         626
                    val           626         811
RFUD_RW3_4_5_6      test          814         995
                    train           0         644
                    val           645         813

In [ ]:
df.to_parquet(
    "canonical_checkpoint_features_with_soh_and_split.parquet",
    index=False
)
